In [15]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR

#From Marks report
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

In [16]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [55]:
from cameo import load_model

infilename = '../../models/iMM904_progesterone_fixed.xml'
print(f"Loading {infilename}")
#model = read_sbml_model(infilename)
model = load_model(infilename)

Loading ../../models/iMM904_progesterone_fixed.xml


In [56]:
# This information is needed to make the model produce progesterone. Otherwise, no progesterone is produced. 
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x07f134775d2e0
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [19]:
# Finding the optimal fluxes through each reaction when the objective is biomass production 
Fluxes=model.optimize()
Fluxes

,fluxes,reduced_costs
CITtcp,0.00000,-0.000000e+00
13BGH,0.00000,-4.751566e-02
13BGHe,0.00000,1.040834e-17
13GS,0.32667,-6.938894e-18
16GS,0.00000,-9.503132e-02
...,...,...
R07215,0.00000,-6.612596e-02
R04804,0.00000,-1.332418e+00
R_methylpentanal_t,0.00000,0.000000e+00
SK_methylpentanal_e,0.00000,0.000000e+00


In [20]:
#Choose the reaction that produced progesterone as the objective.
model.objective = model.reactions.R02216

In [58]:
# Calculate the progesterone yield from glucose as substrate
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)

#optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [59]:
#optswap

In [61]:
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors seapped.
swap_optimization = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["fad_c","fadh2_m"]),objective_function=py)
# VIRKER!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
swap_optimization

In [36]:
# Identify the reactions that when swapped to another co-factor improve the production of progesterone
solution=swap_optimization.run(max_size=20)

Starting optimization at Fri, 18 Nov 2022 11:02:40


HBox()

Finished after 00:00:52


In [37]:
solution

,index,targets,fitness
0,0,"(R02216, GAPD)",0.561771
1,1,"(GAPD,)",0.519898
2,5,"(ALCD23xi, ALCD2ir, ALCD2x_copy2, R02216, BTDD...",0.414508
3,9,"(G3PD1ir, ALCD2ir, ALCD2x_copy2, GLUSx, MDH)",0.313230
4,10,"(ALCD2x_copy2, R02216)",0.237624
5,11,"(NADH2_u6cm, R02216)",0.237624
6,13,"(R02216, C3STDH1, C3STDH2, LNS14DMx)",0.237624
7,18,"(R02216, GLUSx)",0.234146
8,19,"(R02216, LNS14DMx)",0.230769
9,20,"(GLUSx, R02216)",0.230769


In [38]:
reactions=solution.data_frame.targets
reactions

0                                        (R02216, GAPD)
1                                               (GAPD,)
2     (ALCD23xi, ALCD2ir, ALCD2x_copy2, R02216, BTDD...
3          (G3PD1ir, ALCD2ir, ALCD2x_copy2, GLUSx, MDH)
4                                (ALCD2x_copy2, R02216)
5                                  (NADH2_u6cm, R02216)
6                  (R02216, C3STDH1, C3STDH2, LNS14DMx)
7                                       (R02216, GLUSx)
8                                    (R02216, LNS14DMx)
9                                       (GLUSx, R02216)
10                                      (ALCD2x_copy2,)
11                                      (ALCD2x_copy1,)
12                                        (NADH2_u6cm,)
13                                          (ALCD22xi,)
14                                  (C3STDH2, LNS14DMx)
15                                             (HSDxi,)
Name: targets, dtype: object

In [44]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + Nicotinamide adenine dinucleotide - reduced + H+

Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
2 methylpropanal C4H8O + H+ + Nicotinamide adenine dinucleotide - reduced ==> Isobutyl alcohol C4H10O + Nicotinamide adenine dinucleotide

Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced ==> Ethanol + Nicotinamide adenine dinucleotide

Ethanol + Nicotinamide adenine dinucleotide ==> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced

pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + Nicotinamide adenine dinucleotide - reduced + H+

 R R  2 3 Butanediol C4H10O2 + Nicotinam